# 游戏贴图资源自适应超采样和锐化

In [5]:
import os
from PIL import Image, ImageFilter

In [6]:
INPUT_DIR = './Res/TextureResizer/Input'
OUTPUT_DIR = './Res/TextureResizer/Output'
TARGET_SIZE = (200, 200)  # Target size for the images
SHARPEN = True  # Whether to apply sharpening filter

In [7]:
def process_image(image_path, output_path):
    with Image.open(image_path) as img:
        # 计算最佳缩小比率
        ratio = max(img.size[0]/TARGET_SIZE[0], img.size[1]/TARGET_SIZE[1])
        # 分阶段缩小（保护像素特征）
        while ratio > 2:
            next_step = max(int(img.size[0]/2), int(img.size[1]/2))
            img = img.resize((next_step, next_step), resample=Image.LANCZOS)
            ratio = max(img.size[0]/TARGET_SIZE[0], img.size[1]/TARGET_SIZE[1])
        # 最终精确缩放
        img = img.resize(TARGET_SIZE, resample=Image.LANCZOS)
        # 自适应锐化（基于图像内容）
        if SHARPEN:
            # 计算锐化强度（基于尺寸变化比例）
            sharpen_intensity = min(3.0, (img.size[0] / TARGET_SIZE[0]) * 0.5)
            
            # 非均匀锐化掩模
            img = img.filter(ImageFilter.UnsharpMask(
                radius=2,
                percent=int(150 * sharpen_intensity),
                threshold=3
            ))
        # 保存处理后的图像
        img.save(output_path, optimize=True, quality=95)

In [ ]:
#遍历输入目录中的所有图像文件
for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)
        # 创建输出目录（如果不存在）
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        # 处理图像
        process_image(input_path, output_path)
        print(f"Processed {filename} -> {output_path}")

Processed Frozenlake.png -> ./Res/TextureResizer/Output\Frozenlake.png
Processed Permafrost.png -> ./Res/TextureResizer/Output\Permafrost.png
Processed Shrub.png -> ./Res/TextureResizer/Output\Shrub.png
Processed Tundra.png -> ./Res/TextureResizer/Output\Tundra.png


: 